# Uses interaction to push triples to the brain and query it

This notebook assumes you already understand the following notebooks:

* lets-chat.ipynb
* roboGrasp-api.ipynb

In this notebook, we will combine the interaction modeled through EMISSOR with the interaction throught *capsules* with the BRAIN. As auxiliary modules, we will use the *cltl-knowledgeextraction* and *cltl-language-generation*. The *cltl-knowledgeextraction* will extract triples from the utterances either for posting or for querying. In the former case, it also extracts the source perspective from the text. The triples and source perspectives are represented in enriched capsules. We will also use a replier that is include in the *cltl-languagegeneration* package. This replier transfers the response from the BRAIN into natural language expressions and possibly gestures. 

In order to connect to the EMISSOR scenario, we need to align the meta properties of the scenario with the meta data in the capsules. However,  the *cltl-knowledgeextraction* uses a similar data object to keep track of the conversation history. This is a Chat object that needs to be created. Through this Chat object, we keep track of what was said before and deal with coreference to earlier utterances.

Combining EMISSOR and the BRAIN, we can model the interaction in a infite while loop where we go through the following steps:

* We create an EMISSOR scenario at the start of the interaction and a corresponding Chat object to keep track of the dialogue history
* while the user does not stop:
    * create TextSignals for each utterance from the user and store these in an EMISSOR scenario
    * add the utterance also to the Chat instance
    * process the latest utterance in the Chat object using the *cltl-knowledgeextraction* to get triples and perspectives
    * create a capsule from the triples, perspectives and the text signal meta data
    * post the triple to the BRAIN
    * get the answer or throughts as a response from the BRAIN
    * verbalise the answer or thoughts
    * create a new TextSignal for the system response and add it to the EMISSOR scenario
* When the user stops, we save the scenario in the EMISSOR format.



Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  /Users/piek/PycharmProjects/cltl-chatbots/data/2021-11-12-12:16:47  Created 
Directory  /Users/piek/PycharmProjects/cltl-chatbots/data/2021-11-12-12:16:47/image  Created 


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [4]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-11-12 12:16:51,128 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-12 12:16:53,070 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [5]:
replier = LenkaReplier()

2021-11-12 12:16:54,116 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Establish the speaker identity

In [6]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario, AGENT)

print("Name:", HUMAN_NAME)
print("Id:", HUMAN_ID)
friend.add_new_name_to_brain (scenario, 
                  place_id, 
                  location, 
                  HUMAN_ID,
                  textSignal,
                  HUMAN_NAME,
                  my_brain)

Leolani2: Hi! I would like to know your name! Stranger?



 Fred


Fred
Leolani2: So your name is Fred?



 yes


Name: Fred
Id: Fred_t_956924
2021-11-12 12:17:00,076 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred-t-956924_label_fred [person_->_object])
2021-11-12 12:17:00,092 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-11-12 12:17:01,731 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred-t-956924 on November,2021 about UNDERSPECIFIED
2021-11-12 12:17:01,765 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. experience touch - 15 gaps as object: e.g. dislike agent
2021-11-12 12:17:01,777 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 0 gaps as subject: e.g. '' - 2 gaps as object: e.g. own agent
Name capsule: {'chat': '2021-11-12-12:16:47', 'turn': '6f60596c-5117-4d43-85ff-408db28f6f68', 'author': 'fred-t-956924', 'utterance': 'Hi! I would like to know your name! Stranger?', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'pos

{'response': '204',
 'statement': {'chat': '2021-11-12-12:16:47',
  'turn': '6f60596c-5117-4d43-85ff-408db28f6f68',
  'author': 'fred-t-956924',
  'utterance': 'Hi! I would like to know your name! Stranger?',
  'utterance_type': <UtteranceType.STATEMENT: 0>,
  'position': '0-45',
  'subject': {'label': 'Fred_t_956924', 'type': ['person']},
  'predicate': {'type': 'label'},
  'object': {'label': 'Fred', 'type': ['object']},
  'context_id': 'Leolani2',
  'date': datetime.date(2021, 11, 12),
  'place': 'Amsterdam',
  'place_id': 238,
  'country': 'NL',
  'region': 'North Holland',
  'city': 'Amsterdam',
  'objects': [{'type': 'chair', 'confidence': 0.59, 'id': 1},
   {'type': 'table', 'confidence': 0.73, 'id': 1},
   {'type': 'pillbox', 'confidence': 0.32, 'id': 1}],
  'people': [{'name': 'Fred_t_956924', 'confidence': 0.98, 'id': 1}],
  'triple': fred-t-956924_label_fred [person_->_object]),
  'perspective': <cltl.brain.infrastructure.api.Perspective at 0x7fbfa06ebbd0>,
  'type': <Uttera

## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [7]:
chat = Chat(HUMAN_ID)

2021-11-12 12:19:05,867 -     INFO - cltl.triple_extraction.api.Chat (Fred_t_956924)     000 - << Start of Chat with Fred_t_956924 >>


## Start the interaction

In [ ]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain)
    pprint.pprint(capsule)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)



Leolani2: Tell me anything, I love learning things



 I like pizza


Fred: I like pizza
2021-11-12 12:19:44,890 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-12 12:19:44,891 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-12 12:19:44,897 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-12 12:19:46,017 -     INFO - cltl.triple_extraction.api.Chat (Fred_t_956924)     001 - Fred_t_956924: "I like pizza"
2021-11-12 12:19:47,303 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred-t-956924_like_pizza [agent_->_food])
2021-11-12 12:19:47,341 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-11-12 12:19:49,393 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred-t-956924 on November,2021 about POSITIVE
2021-11-12 12:19:49,445 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. travel-to location - 15 gaps as obje


 Who likes pizza?


Fred: Who likes pizza?
2021-11-12 12:19:59,910 -     INFO - cltl.triple_extraction.api.Chat (Fred_t_956924)     002 - Fred_t_956924: "Who likes pizza?"
2021-11-12 12:19:59,914 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: pizza_like_? [food_->_])
{'author': 'Fred_t_956924',
 'chat': '2021-11-12-12:16:47',
 'city': 'Amsterdam',
 'context_id': 'Leolani2',
 'country': 'NL',
 'date': datetime.date(2021, 11, 12),
 'object': {'label': '', 'type': []},
 'objects': [{'confidence': 0.59, 'id': 1, 'type': 'chair'},
             {'confidence': 0.73, 'id': 1, 'type': 'table'},
             {'confidence': 0.32, 'id': 1, 'type': 'pillbox'}],
 'people': [{'confidence': 0.98, 'id': 1, 'name': 'Fred_t_956924'}],
 'perspective': None,
 'place': 'Amsterdam',
 'place_id': 238,
 'position': '0-16',
 'predicate': {'type': 'like'},
 'region': 'North Holland',
 'subject': {'label': 'pizza', 'type': ['noun.food']},
 'triple': pizza_like_? [food_->_]),
 'turn': 'e438bb74-8e06-4bf0-

...

None
Leolani2: So let it be written, ... so let it be done



 What Fred likes?


Fred: What Fred likes?
2021-11-12 12:21:44,135 -     INFO - cltl.triple_extraction.api.Chat (Fred_t_956924)     006 - Fred_t_956924: "What Fred likes?"
2021-11-12 12:21:44,139 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: likes_Fred_? [cognition_->_])
{'author': 'Fred_t_956924',
 'chat': '2021-11-12-12:16:47',
 'city': 'Amsterdam',
 'context_id': 'Leolani2',
 'country': 'NL',
 'date': datetime.date(2021, 11, 12),
 'object': {'label': '', 'type': []},
 'objects': [{'confidence': 0.59, 'id': 1, 'type': 'chair'},
             {'confidence': 0.73, 'id': 1, 'type': 'table'},
             {'confidence': 0.32, 'id': 1, 'type': 'pillbox'}],
 'people': [{'confidence': 0.98, 'id': 1, 'name': 'Fred_t_956924'}],
 'perspective': None,
 'place': 'Amsterdam',
 'place_id': 238,
 'position': '0-16',
 'predicate': {'type': 'Fred'},
 'region': 'North Holland',
 'subject': {'label': 'likes', 'type': ['noun.cognition']},
 'triple': likes_Fred_? [cognition_->_]),
 'turn': '0167


 Fred likes what?


Fred: Fred likes what?
2021-11-12 12:22:06,777 -     INFO - cltl.triple_extraction.api.Chat (Fred_t_956924)     007 - Fred_t_956924: "Fred likes what?"
2021-11-12 12:22:06,839 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of fred to: object
2021-11-12 12:22:06,907 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_like_what [_->_agent or object])
2021-11-12 12:22:06,990 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - existing object 
2021-11-12 12:22:09,640 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred-t-956924 on November,2021 about POSITIVE
2021-11-12 12:22:09,695 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 0 gaps as subject: e.g. '' - 2 gaps as object: e.g. own agent
2021-11-12 12:22:09,747 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 16 gaps as subject: e.g. study-at institution - 11 gaps as object: e.g. like-by agent
{'a

## Save the scenario data

In [14]:
scenario.scenario.ruler.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook